# Federated collaborative filtering 

Your task is to implement federated collaborative filterng model for privacy-preserving personalized recommendation system.  
In the Federated Learning paradigm, a master machine learning model is distributed to user clients, the clients use their locally stored data and model for both inference and calculating model updates. The model updates are sent back and aggregated on the server to update the master
model then redistributed to the clients. In this paradigm, the user data never leaves the client, greatly enhancing the user’ privacy, in contrast to the traditional paradigm of collecting, storing and processing user data on a backend server beyond the user’s control. The federated updates to the model are based on a gradient approach. 

Specifically, the method defines three core components as below.
1. All the item factor vectors $y_i, i = 1, ... , M$ are updated on the server and then distributed to each client $u$.
2. The user factor vectors $x_u, u = 1, ... , N$ are updated locally on the client $u$, using the user $u$’s own data and the $y_i, i = 1, ... , M$ from the server.
3. The updates through the gradients $\delta y_{ui}$ are calculated for the item $i$ on each client $u$ and transmitted to the server where the gradients are aggregated to update $y_i$.

The cost function optimizing across all users $u$ and the items $i$ over the confidence levels $c_{ui}$ is then given as

$$J = \sum_u \sum_i c_{ui}(p_{ui} - x_u^T y_i)^2 + \lambda (\sum_u||x_u||^2 + \sum_i ||y_i||^2),$$
with a regularization parameter $\lambda$,
where 
$$p_{ui} = \begin{cases}  
1 & r_{ui} > 0,\\
0 & r_{ui} = 0
\end{cases}$$
 $$c_{ui} = 1 + \alpha r_{ui}$$

User factors are updated by directly solving an equation

$$ \frac{\partial J(x_u)}{\partial x_u} = 0 $$
Thus, we obtain the solution
$$ x_u = (YC^uY^T + \lambda I)^{-1}YC^up(u),$$
where $ C^u \in \mathbb{R}^{N \times N}$ is a diagonal matrix with $C_{ii}^u = c_{ui}$ and $p(u) \in \mathbb{R}^{N \times 1}$ contains the $p_{ui}$ values for the user $u$.

Item factors are updated on the master server as

$$ y_i = y_i - \gamma \frac{\partial J}{\partial y_i},$$

where gamma is gain parameter.

$$ \frac{\partial J}{\partial y_i} = -2 \sum_u f(u,i) +2 \lambda y_i, $$
where 
$$ f(u,i) = \left[ c_{ui}(p_{ui} - x_u^T y_i)\right]x_u.$$

To familiarise yourself with the method, read the paper https://arxiv.org/pdf/1901.09888.pdf


In [1]:
#!pip install ipypb

In [2]:
#!pip install --upgrade git+https://github.com/evfro/polara.git@develop#egg=polara

In [3]:
from scipy.sparse import csr_matrix
from ipypb import irange

In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from polara import get_movielens_data
from polara.evaluation.pipelines import random_grid, find_optimal_config

In [5]:
from dataprep import transform_indices, verify_time_split
from evaluation import topn_recommendations, model_evaluate, downvote_seen_items

## Download dataset

For this task we will use popular MovieLens dataset.

In [6]:
data = get_movielens_data(
    get_genres=False, # we won't need genres
    split_genres=False,
    include_time=True
)

data.head()

,userid,movieid,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# 1. Preprocess data  (5 pts)
Do the preprocessing, including data split and index transformation. You are recommended to use functions from the seminar on evaluation.

## Data split 


Split data to train/validation/test datasets, using leave-last-out scheme for holdout sampling.

In [7]:
def leave_last_out(data, userid='userid', timeid='timestamp'):
    data_sorted = data.sort_values('timestamp')
    holdout = data_sorted.drop_duplicates(
        subset=['userid'], keep='last'
    ) # split the last item from each user's history
    remaining = data.drop(holdout.index) # store the remaining data - will be our training
    return remaining, holdout

In [8]:
training_, holdout_ = leave_last_out(data)

In [9]:
training_, holdout_val_ = leave_last_out(training_)

## Transform index

Transform index in train dataset and reindex validation and test datasets.

In [10]:
def transform_indices(data, users, items):
    data_index = {}
    for entity, field in zip(['users', 'items'], [users, items]):
        new_index, data_index[entity] = to_numeric_id(data, field)
        data = data.assign(**{f'{field}': new_index}) # makes a copy of dataset!
    return data, data_index

def to_numeric_id(data, field):
    idx_data = data[field].astype("category")
    idx = idx_data.cat.codes
    idx_map = idx_data.cat.categories.rename(field)
    return idx, idx_map

def reindex_data(data, data_index, fields=None):
    if fields is None:
        fields = data_index.keys()
    if isinstance(fields, str): # handle single field provided as a string
        fields = [fields]
    for field in fields:
        entity_name = data_index[field].name
        new_index = data_index[field].get_indexer(data[entity_name])
        data = data.assign(**{f'{entity_name}': new_index}) # makes a copy of dataset!
    return data

In [11]:
train_data, data_index = transform_indices(training_, 'userid', 'movieid')

In [12]:
test_data = reindex_data(holdout_, data_index)
val_data = reindex_data(holdout_val_, data_index)

In [13]:
test_data = test_data.query('movieid >= 0')

In [14]:
# sorting data by user id for correct evaluation
test_data = test_data.sort_values('userid') 
val_data = val_data.sort_values('userid')

In [15]:
data_description = dict(
    users = data_index['users'].name, # user field
    items = data_index['items'].name, # item field
    test_users = test_data[data_index['users'].name].values,
    holdout_items = test_data[data_index['items'].name].values,
    val_users = val_data[data_index['users'].name].values,
    holdout_val_items = val_data[data_index['items'].name].values,
    n_users = train_data[data_index['users'].name].nunique(), 
    n_items = train_data[data_index['items'].name].nunique()
)

# 2. Building/training a recommender model (20 pts)

## Preparing data in matrix form

Complete the function to obtain sparse matrix from training data.

In [16]:
def matrix_from_observations(data, userid, itemid, feedback=None, shape=None, dtype=None):
    '''Return sparse matrix, obtained from training data'''
    useridx = data[userid]
    itemidx = data[itemid]
    values = data[feedback]
    return csr_matrix((values, (useridx, itemidx)))   

In [17]:
data_matrix = matrix_from_observations(train_data, userid='userid', itemid='movieid', feedback='rating')
data_matrix

<6040x3704 sparse matrix of type '<class 'numpy.int64'>'
	with 988129 stored elements in Compressed Sparse Row format>

## The model

In [18]:
n_users = data_description['n_users']
n_items = data_description['n_items']
n_factors = 50
seed = 42

In [19]:
# initialization of user and item factors
random_state = np.random.RandomState(seed)
user_factors = random_state.normal(0, 0.01, size=(n_users, n_factors))
item_factors = random_state.normal(0, 0.01, size=(n_items, n_factors))

## Server side

Complete the functions to update item factors.


In [20]:
def update_global_model(matrix, X, Y, regularization, gain, alpha, n_iter):
    grad = np.zeros(Y.shape)
    for _ in range(n_iter):
        grad += item_factor_update(matrix, X, Y, regularization, alpha, n_iter)
    Y -= gain * grad # update Y


def item_factor_update(matrix, X, Y, regularization, alpha, n_iter=4):
    '''Update item factor vectors'''
    n_users , _ = X.shape
    grad = 2 * regularization * Y # initialize gradient as a regularization term
    for u in range(n_users):
        grad += fetch_neg_client_grad(matrix, X, Y, u, alpha) # add calculated gradient for a user
    return grad
    

def fetch_neg_client_grad(matrix, X, Y, u, alpha):
    '''
    Fetches for entire item collection at once
    f(u, i) = c_ui * (p_ui - yi^T xu) xu
    f(u, Y) = C_u *  (p_u  -   Y xu ) xu^T = - (Y xu + (Cu-1)*(Yxu - pu)) xu^T
    '''
    indptr = matrix.indptr # get indptr from sparse matrix
    inds = matrix.indices # get indices from sparse matrix
    r = matrix.data # get ratings
    ru = np.zeros((matrix.shape[1])) 
    for i in range(indptr[u], indptr[u + 1]):
        ru[inds[i]] = r[i]
    Cu_ = alpha * ru
    pu = np.zeros_like(ru)
    pu[ru > 0] = 1
    
    xu = X[u] # get user's vector  
    user_neg_grad_val = (Y @ xu - pu + Cu_ * (Y @ xu - pu)).reshape((-1, 1)) @ xu.reshape((1, -1)) 
    return 2 * user_neg_grad_val

## Client side

Complete the functions to update user factors.

In [21]:
def update_local_models(matrix, X, Y, regularization, alpha):
    n_users, n_factors = X.shape
    YtY = Y.T @ Y + regularization * np.eye(n_factors)
    for u in range(n_users): # to imitate locally updating on the client side
        X[u] = user_factor_update(Y, YtY, matrix, u, alpha) # each client does independently


def user_factor_update(Y, YtY, matrix, u, alpha):
    '''
    Return the solution of equation
    Xu = (YtCuY + regularization * I)^-1 (YtCuPu)
    '''
    A, b = solve_linear_equation(Y, YtY, matrix, u, alpha)
    return np.linalg.inv(A) @ b


def solve_linear_equation(Y, YtY, matrix, u, alpha):
    '''
    Return A = YtCuY + regularization * I and b = YtCuPu
    to use it later for solving the equation
    Xu = (YtCuY + regularization * I)^-1 (YtCuPu)

    Following identity might be helpful
    YtCuY + regularization * I = YtY + regularization * I + Yt(Cu-I)Y
    '''
    indptr = matrix.indptr # get indptr from sparse matrix 
    inds = matrix.indices # get indices from sparse matrix
    r = matrix.data # get ratings
    
    ru = np.zeros((matrix.shape[1]))
    for i in range(indptr[u], indptr[u + 1]):
        ru[inds[i]] = r[i]
    Cu_ = alpha * ru
    pu = np.zeros_like(ru)
    pu[ru > 0] = 1
    
    A = YtY + (Y.T * Cu_) @ Y # accumulate YtCuY + regularization * I in A
    b = Y.T @ pu + Y.T @ (Cu_ * pu) # accumulate YtCuPu in b
    return A, b  

## Model Training
Complete the function to build model. Choose the hyper-parameters and store it in config. Train your model on chosen hyper-parameters.

In [22]:
def build_model(model_config, trainset, trainset_description):
    n_factors = model_config['n_factors']
    n_epochs = model_config['n_epochs']
    regularization = model_config['regularization']
    gain = model_config['gain']
    alpha = model_config['alpha']
    n_iter = model_config['n_iter']
    seed = model_config['seed']
    
    n_users = trainset_description['n_users']
    n_items = trainset_description['n_items']
    
    train_matrix = trainset # get sparse matrix
    random_state = np.random.RandomState(seed)
    user_factors = random_state.normal(0, 0.01, size=(n_users, n_factors)) # initialize user factors
    item_factors = random_state.normal(0, 0.01, size=(n_items, n_factors)) # initialize user factors

    for epoch in tqdm(range(n_epochs)):
        update_global_model(train_matrix, user_factors, item_factors, regularization, gain, alpha, n_iter)
        update_local_models(train_matrix, user_factors, item_factors, regularization, alpha)
        # in each epoch update global model and then update local models
            
    return user_factors, item_factors

In [23]:
# create config with the following hyper parameters:
# seed, n_factors, regularization, gain, alpha, n_epochs, n_iter.
config = {
    'seed':0,
    'n_factors':50,
    'regularization':1.0,
    'gain':0.05,
    'alpha':1.0,
    'n_epochs':20,
    'n_iter':1
}

In [24]:
params = build_model(config, data_matrix, data_description) 

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [06:47<00:00, 20.39s/it]


In [25]:
params[0].shape

(6040, 50)

# 3. Evaluation (5 pts)

Complete the functions to predict relevance scores. Using predcted scores generate top-n recommendations. Evaluate quality of recommendations and report HR@5 and MRR@5 etrics. Remember that we don't want to recommend items from the user's history. You are recommended to use functions from the previous seminars.

In [26]:
topn = 5

In [27]:
test_data

,userid,movieid,rating,timestamp
25,0,47,5,978824351
66,1,1550,3,978300174
232,2,1900,4,978298504
237,3,971,4,978294282
258,4,279,2,978246585
...,...,...,...,...
999251,6035,2440,1,956755196
999684,6036,421,3,956801840
999731,6037,1094,5,956717204
999826,6038,1162,4,956758029


In [47]:
# Complete model scoring function to predict relevance scores for test user-item pairs.
def model_scoring(model_params, testset, testset_description):
    user_factors, item_factors = model_params # get user and item factros
    test_user_factors = user_factors[testset.userid.unique()] # get user factros for test users
    scores = test_user_factors @ item_factors.T # get scores 
    print(scores.shape)
    downvote_seen_items(scores, testset, data_description)
    return scores

In [48]:
# Complete model recommendations function to generate top-n recommendations using predcted scores.
def model_recom_func(model_scores, topn=5):
    recommendations = topn_recommendations(model_scores, topn)
    return recommendations

In [49]:
# Complete function to evaluate quality of recommendations
def evaluate_func(model_recoms, holdout, holdout_description, topn=5):
    hr, mrr, _ = model_evaluate(model_recoms, holdout, holdout_description, topn)
    return hr, mrr

## Evaluate model

In [50]:
userid = data_description['users']
seen_idx_mask = train_data[userid].isin(data_description['test_users'])
testset = train_data[seen_idx_mask]

In [51]:
test_scores = model_scoring(params, testset, data_description) # get scores
recoms = model_recom_func(test_scores) # get recommended items
hr, mrr = evaluate_func(recoms, test_data, data_description) # get metrics
print(f'HR@{topn} = {hr:.4f}')
print(f'MRR@{topn} = {mrr:.4f}')

(6038, 3704)
HR@5 = 0.0366
MRR@5 = 0.0173


# Quick check

Compare your model with one or two baselines and report the results. You can use code from previous seminars. 

In [ ]:
...

# 4. Hyper-parameter selection (5 pts)

Complete the function to tune hyper-parameters of your model to provide the best quality. Train the model with the best hyper-parameters and report HR and MRR on test dataset.

In [94]:
# you are free to change hp values
params_dict = dict(
    n_factors = [10, 20, 50, 100],
    regularization = np.linspace(1, 5, 5).astype(int),
    gain = np.logspace(-4, -2, 5),
    alpha = np.linspace(1, 5, 5).astype(int),
    n_epochs = [10, 15, 20, 25],
    n_iter = np.linspace(4, 20, 5).astype(int),
    seed = [0]
)

In [95]:
param_grid, param_names = random_grid(params_dict, n=30)

In [107]:
def hp_selection(data, description, param_grid, param_names, testset, test_data):
    ''' Return best HR value and best set of hyperparameters '''
    data_matrix = matrix_from_observations(data, userid='userid', itemid='movieid', feedback='rating')
    best_hr = 0
    best_params = None
    for parameter_model in param_grid:
        #print(param_names, parameter_model)
        configuration = dict(zip(param_names, parameter_model))
        model_params = build_model(configuration, data_matrix, data_description) 
    
        hr, mrr = evaluate_func(
            model_recom_func(
                model_scoring(
                    model_params,
                    testset,
                    data_description
                )
            ), test_data,
            data_description
        )
        
        if hr > best_hr:
            best_hr = hr
            best_params = config
    return best_hr, best_params

In [108]:
# Train the model with the best hyper-parameters and report HR and MRR on test dataset.
best_hr, best_params = hp_selection(train_data, data_description, param_grid, param_names, testset, test_data)
print(f'HR@{topn} = {hr:.4f}')
print(f'MRR@{topn} = {mrr:.4f}')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [38:38<00:00, 92.73s/it]


(6038, 3704)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [23:08<00:00, 92.56s/it]


(6038, 3704)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [16:50<00:00, 40.44s/it]


(6038, 3704)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [15:40<00:00, 94.07s/it]


(6038, 3704)


 88%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▍              | 22/25 [24:40<03:21, 67.29s/it]


KeyboardInterrupt: 

# 5. Analysis (5 pts) 

Examine how the loss function, $||X||$ and $||Y||$ change during the training. Complete the function to obtain the history of losses values, $||X||$ and $||Y||$ for every epoch. Use best hyper-parameters for model config. Plot the graphs and discuss the results.

In [ ]:
def model_analysis(model_config, trainset, trainset_description):
    ...    
    return X_norms, Y_norms, losses

In [ ]:
X_norms, Y_norms, losses = model_analysis(best_params, train_data, data_description)

In [ ]:
# Plot the graphs and discuss the results.
...

What is the time complexity of the federated collaborative filterng with gradient descent? 

In [112]:
from tqdm.notebook import tqdm

for i in tqdm(range(1000000000)):
    pass

  0%|          | 0/1000000000 [00:00<?, ?it/s]

KeyboardInterrupt: 